In [1]:
import pandas as pd
import unicodedata
from bs4 import BeautifulSoup
import requests
from csv import writer
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
driver = webdriver.Chrome("../chromedriver")

/var/folders/g8/7k8r16bd1777rydq47tft1240000gn/T/ipykernel_26665/3092556330.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("../chromedriver")


In [3]:
# some codes are from: 
# https://medium.com/analytics-vidhya/how-to-scrape-a-table-from-website-using-python-ce90d0cfb607

# home url of top scores
home_url = 'https://www.topscores.co/Vic/vce-school-rank-median-vce/2021/'

# 25 of 654 entries are shown per page, so in total we have 27 pages of table contains all the schools
page_numbers = list(range(28))[1:28]

#create a datafram to store the data
headers = ["rank", "School", "Location", "MedianVCE Score", "VCE40+%"]
school_rank = pd.DataFrame(columns = headers)


count = 0
for page in page_numbers:
    print(page)
    
    # first page has a different url with other pages so we do this seperately
    if count == 0:
        url = home_url      
    else:
        # get url for 2 to 27 pages
        url = home_url + "?pageno=" + str(page)
    
   
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    
    #for each page find the table
    table = soup.find('table', attrs={'class': 'reportTable'})
    
    # Create a for loop to scrap each row from the table
    for j in table.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        
        # to filter out the ad windows
        if row[0].isdigit():
            length = len(school_rank)
            school_rank.loc[length] = row
        # skip the ad windows
        else:
            continue
    
    
    count = count + 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [4]:
school_rank

,rank,School,Location,MedianVCE Score,VCE40+%
0,1,Haileybury Girls College,Keysborough,37,37.2
1,2,Bialik College,Hawthorn East,37,32.6
2,3,Ballarat Clarendon College,Ballarat,37,32.5
3,4,St Catherines School,Toorak,37,32.2
4,5,Huntingtower School,Mount Waverley,37,31.6
...,...,...,...,...,...
649,650,VSL Traralgon-Brentwood-Haileybury-Mentone,Thornbury,,
650,651,VSL University-Brunswick-Collingwood,Thornbury,,
651,652,VSL Westall-South Oakleigh-McKinnon,Thornbury,,
652,653,Western Chinese Language School Braybrook,Braybrook,,


In [12]:
# save the data
school_rank.to_csv('school_rank.csv', index=False)